In [37]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from math import pi
from matplotlib.lines import Line2D
import tomllib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score
import random
import warnings
import matplotlib as mpl

from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import copy

In [308]:
def dew_point_cacl(t, h):
    """t- температура в градусах цельсия, h - влажность воздуха в виде ХХ"""
    a, b = 17.27, 237.7
    g = a * t / (b + t)
    h = np.clip(h, 1e-6, 100.0)
    g += np.log(h / 100)
    dp = (b * g) / (a - g)
    return dp

In [309]:
ugv = pd.read_feather("ugv.feather")
meteo = pd.read_feather("meteo.feather")
meteo["dew_point"] = dew_point_cacl(
    meteo["temp_air"].to_numpy(), meteo["hum_air"].to_numpy()
)
alldata = meteo.copy()

In [310]:
curent_time = alldata["datetime"].max() - pd.Timedelta(days=2)
data_timesort = alldata[alldata["datetime"] > curent_time]

In [311]:
data = data_timesort[data_timesort["id_s"] == "5"].reset_index()
data["temp_air"] = data["temp_air"]+8
data["hum_air"] = data["hum_air"] + 30
data["dew_point"] = dew_point_cacl(
    data["temp_air"].to_numpy(), data["hum_air"].to_numpy()
)

работа с toml

In [324]:
class Diseas_work:
    def __init__(self):
        self.names = []
        self.current = []
        self.diseas = {}
        self.time = {}
        self.valid={}

    def add_diseas_toml(self, file, coef=0):
        with open(file, "rb") as f:
            diseases = tomllib.load(f)
        for d_name in diseases:
            self.names.append(d_name)
            self.diseas[d_name] = pd.DataFrame(columns=("min", "max"))
            for param in diseases[d_name]["param"]:
                if param == "time":
                    self.time[d_name]=diseases[d_name]["param"]['time']['min']
                elif param =='dew_point': continue
                else:
                    delta = (
                        (diseases[d_name]["param"][param]["max"] - diseases[d_name]["param"][param]["min"])
                        / 100* coef
                    )
                    self.diseas[d_name].loc[param] = [
                        diseases[d_name]["param"][param]["min"] - delta,
                        diseases[d_name]["param"][param]["max"] + delta,
                    ]
            if 'dew_point' in diseases[d_name]["param"]:
                # print('da')
                # print(self.diseas[d_name].loc['temp_air'].to_numpy())
                self.diseas[d_name].loc['dew_point']=dew_point_cacl(self.diseas[d_name].loc['temp_air'].to_numpy(),self.diseas[d_name].loc['hum_air'].to_numpy())


    def compair_vaues(self, data, diseas_name):
        df_range = copy.copy(self.diseas[diseas_name])
        df_values = copy.copy(data)
        common_params = df_values.columns.intersection(df_range.index)
        df_range = df_range.reindex(common_params)
        df_values = df_values[common_params]
        valid = (df_values >= df_range["min"]) & (df_values <= df_range["max"])
        valid['all'] = valid.all(axis=1)
        valid['datetime']=data['datetime']
        return valid

    def check_time(self, data, time="last"):
        if time == "last":
            time_last = data["datetime"].max()
        elif time == "all":
            time_last = data["datetime"].min()
        else:
            time_last = data["datetime"].max()-pd.Timedelta(hours=time)
        data_last = data[data["datetime"] >= time_last]
        for d_name in self.names:
            print("check ", d_name)
            valid = self.compair_vaues(data_last, d_name)
            # print(valid)
            self.valid[d_name]=valid
            # if valid.all().all():
            #     print('ALARM', d_name)
            #     self.diseas_current.append(d_name)




In [325]:
d=Diseas_work()
d.add_diseas_toml('haze.toml',0)
d.check_time(data,'all')
d.valid[d.names[0]]

check  Mildew
check  Oidium
check  Botrytis
check  BlackRot
check  Anthracnose
check  Drought
check  Frost
check  Sunburn


,temp_air,hum_air,dew_point,all,datetime
0,False,False,False,False,2025-10-21 05:13:04
1,False,False,False,False,2025-10-21 05:40:50
2,False,False,False,False,2025-10-21 05:56:16
3,False,False,False,False,2025-10-21 06:11:32
4,False,False,False,False,2025-10-21 06:35:12
...,...,...,...,...,...
82,True,True,True,True,2025-10-22 09:49:35
83,True,True,True,True,2025-10-22 10:04:52
84,True,True,True,True,2025-10-22 10:20:15
85,True,True,True,True,2025-10-22 10:35:33


In [319]:
d.diseas[d.names[0]]

,min,max
temp_air,19.000000,26.0
hum_air,76.000000,104.0
dew_point,14.669053,26.0


In [278]:
d.valid[d.names[0]]

,temp_air,hum_air,dew_point,all,datetime
0,False,False,True,False,2025-10-21 05:13:04
1,False,False,True,False,2025-10-21 05:40:50
2,False,False,True,False,2025-10-21 05:56:16
3,False,False,True,False,2025-10-21 06:11:32
4,False,False,True,False,2025-10-21 06:35:12
...,...,...,...,...,...
82,True,True,True,True,2025-10-22 09:49:35
83,True,True,True,True,2025-10-22 10:04:52
84,True,True,True,True,2025-10-22 10:20:15
85,True,True,True,True,2025-10-22 10:35:33


In [41]:
for diseas in diseases_current:
    time_range = diseases[diseas]["param"]["time"]["min"]
    time_last = data["datetime"].max()-pd.Timedelta(hours=time_range)
    data_last = data[data["datetime"] >= time_last]
    parameters = list(diseases[diseas]["param"])
    parameters.remove("time")
    print(data_last)
    print(diseas)
    print(parameters)
    count=0
    for i in range(-1,-len(data_last),-1):
        # 
        count=0
        for param in parameters:
            pmin = diseases[diseas]["param"][param]["min"]
            pmax = diseases[diseas]["param"][param]["max"]
            boud=(pmax-pmin)/10
            if pmin-boud <= data_last.iloc[i][param] <= pmax+boud:
                count+=1
                print(param, pmin, pmax, data_last.iloc[i][param])
        if count==len(parameters):
            time_start=data_last.iloc[i]['datetime']
            print('время началось', i, time_start)


     index id_s  batt  rssi            datetime  temp_air  hum_air  pres_hpa  \
87   16831    5    10   -85 2025-10-22 06:12:04      13.0    110.0     991.4   
88   16834    5    21   -86 2025-10-22 06:27:27      13.6    108.0     991.4   
89   16837    5    10   -86 2025-10-22 06:42:44      12.9    111.0     991.3   
90   16840    5    12   -86 2025-10-22 06:58:07      13.7    108.0     991.5   
91   16843    5    12   -85 2025-10-22 07:13:24      14.1    106.0     991.5   
92   16846    5    10   -85 2025-10-22 07:28:47      14.6    105.0     991.3   
93   16849    5    10   -86 2025-10-22 07:44:04      14.8    104.0     991.6   
94   16852    5    11   -86 2025-10-22 07:59:28      15.4    103.0     991.6   
95   16855    5     7   -86 2025-10-22 08:18:56      15.6    104.0     991.9   
96   16858    5     6   -87 2025-10-22 08:34:24      16.9    100.0     992.0   
97   16863    5     6   -86 2025-10-22 08:58:07      17.5     98.0     991.6   
98   16868    5     5   -86 2025-10-22 0